In [ ]:
import os
import pandas as pd
import koreanize_matplotlib

data_folder = 'data'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# KGUWTHRDLVRDF 파일(컬럼명 있는 파일) 찾기
col_file = None
for f in csv_files:
    if 'KGUWTHRDLVRDF' in f:
        col_file = f
        break

# 컬럼명 추출
if col_file is not None:
    col_file_path = os.path.join(data_folder, col_file)
    col_names = pd.read_csv(col_file_path, nrows=0).columns.tolist()
else:
    raise ValueError("KGUWTHRDLVRDF 파일을 찾을 수 없습니다.")

df_list = []
for file in csv_files:
    file_path = os.path.join(data_folder, file)
    if file == col_file:
        df_temp = pd.read_csv(file_path)
    else:
        df_temp = pd.read_csv(file_path, header=None, names=col_names)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True)
print(df.shape)
df.head()


In [ ]:
import pandas as pd

# data 폴더 내의 모든 엑셀 확장자 파일(.xls, .xlsx) 찾기
excel_files = [f for f in os.listdir(data_folder) if f.endswith('.xls') or f.endswith('.xlsx')]

excel_df_list = []
for excel_file in excel_files:
    excel_path = os.path.join(data_folder, excel_file)
    df_excel = pd.read_excel(excel_path)
    excel_df_list.append(df_excel)

# 여러 엑셀 파일을 하나의 데이터프레임으로 합치기 (필요시)
if excel_df_list:
    df_excel_all = pd.concat(excel_df_list, ignore_index=True)
    display(df_excel_all.head())
else:
    print("엑셀 파일이 없습니다.")



In [26]:
# df_excel_all에서 컬럼한글명과 컬럼영문명을 가져와서 df의 컬럼명을 컬럼한글명으로 변경
if '컬럼한글명' in df_excel_all.columns and '컬럼영문명' in df_excel_all.columns:
    # 컬럼영문명-컬럼한글명 매핑 딕셔너리 생성
    col_map = dict(zip(df_excel_all['컬럼영문명'], df_excel_all['컬럼한글명']))
    # df의 컬럼명을 한글명으로 변경
    df.rename(columns=col_map, inplace=True)
else:
    print("df_excel_all에 '컬럼한글명' 또는 '컬럼영문명' 컬럼이 없습니다.")

In [ ]:
col_map

In [ ]:
df.info()


In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:

df.tail()

In [ ]:

df.sample(10)
df.isnull().sum()
df.isnull().sum().sum()
df.isnull().sum().sum() / df.shape[0] * 100
df.isnull().sum().sum() / df.shape[0] * 100

In [ ]:
df.hist(bins=50, figsize=(12, 12));

In [ ]:
df["강수 유형명"].value_counts()

In [ ]:
display(df['광역시도명'].value_counts())
display(df['시군구명'].value_counts())
display(df['강수 유형명'].value_counts())
display(df['강수 유형명'].value_counts())

In [ ]:
# 강수 유형명(y) 예측 모델 작성

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 사용할 feature 선택 (object 타입 및 y 제외)
X = df.drop(columns=["강수 유형명", "광역시도명", "시군구명", "날짜"])
y = df["강수 유형명"]

# 범주형 feature 인코딩 (필요시)
X = X.copy()
for col in X.select_dtypes(include="object").columns:
    X[col] = X[col].astype("category").cat.codes

# 결측치 처리 (간단히 0으로 대체)
X = X.fillna(0)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 학습
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 예측 및 평가
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 피처 중요도 추출
importances = clf.feature_importances_
feature_names = X.columns

# 중요도 내림차순 정렬
indices = np.argsort(importances)[::-1]

# 중요도 출력
print("Feature importances:")
for f in range(len(feature_names)):
    print(f"{feature_names[indices[f]]}: {importances[indices[f]]:.4f}")

# 시각화
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(len(feature_names)), importances[indices], align="center")
plt.xticks(range(len(feature_names)), feature_names[indices], rotation=90)
plt.tight_layout()
plt.show()
